In [ ]:
!pip install selenium

In [ ]:
!pip install beautifulsoup4

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import sys
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
driver = webdriver.Chrome('C:/Users/limst/Documents/Computational Social Sciences/Instagram-Comments-Scraper-manual/.venv/bin/chromedriver.exe')
driver.get('https://www.instagram.com/')

username_input = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='username']")))
password_input = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='password']")))

username_input.clear()
password_input.clear()

username_input.send_keys("stephanielsm")
password_input.send_keys("Krystal@156")

log_in = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[type='submit']"))).click()

not_now = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//button[contains(text(),'Not Now')]"))).click()
not_now2 = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//button[contains(text(),'Not Now')]"))).click()

In [35]:
cwd = os.path.abspath('')
files = os.listdir(cwd)
files

['.ipynb_checkpoints',
 'gov.sg_circuit_breaker.csv',
 'gov.sg_circuit_breaker_comments.csv',
 'gov.sg_economic_measures.csv',
 'gov.sg_economic_measures_comments.csv',
 'gov.sg_foreign_workers.csv',
 'gov.sg_foreign_workers_comments.csv',
 'gov.sg_general.csv',
 'gov.sg_general_comments.csv',
 'gov.sg_safe_entry_comments.csv',
 'gov.sg_social_distancing.csv',
 'gov.sg_social_distancing_comments.csv',
 'gov.sg_trace_together.csv',
 'gov.sg_trace_together_comments.csv',
 'gov.sg_vaccination.csv',
 'gov.sg_vaccination_comments.csv',
 'Instagram Manual Scrape.ipynb',
 'leehsienloong_circuit_breaker.csv',
 'leehsienloong_circuit_breaker_comments.csv',
 'leehsienloong_economic_measures.csv',
 'leehsienloong_economic_measures_comments.csv',
 'leehsienloong_foreign_workers.csv',
 'leehsienloong_foreign_workers_comments.csv',
 'leehsienloong_general.csv',
 'leehsienloong_safe_entry.csv',
 'leehsienloong_safe_entry_comments.csv',
 'leehsienloong_social_distancing.csv',
 'leehsienloong_social_di

In [36]:
df = pd.DataFrame()

In [37]:
df = pd.read_csv('leehsienloong_general.csv')
        

In [38]:
df.shape

(411, 6)

In [39]:
def get_comments(url,pubdate,description):
        i = 0
        user_names = []
        user_comments = []
        dates = []
        soup = BeautifulSoup(driver.page_source,'lxml')
        comms = soup.find_all('div',attrs={'class':'C4VMK'})
        soup_2 = BeautifulSoup(str(comms),'lxml')                            
        spans = soup_2.find_all('span',class_="")
        h3 = soup_2.find_all('h3',class_='_6lAjh')
        for comment in spans:
            c = comment.text
            user_comments.append(c)
        for name in h3:
            n = name.div.span.a.text
            user_names.append(n)

        comments = driver.find_elements_by_class_name('gElp9 ')
        for c in comments:
            container = c.find_element_by_class_name('C4VMK')
            date = container.find_element_by_class_name('FH9sR').get_attribute('datetime')
            dates.append(date)
        dates.pop(0)
        user_comments.pop(0)
        data = [user_names,user_comments,dates]
        new = pd.DataFrame(data)
        new = new.transpose()
        new = new.rename(columns={0:'username',1:'comments',2:'timestamp'})
        new['postUrl'] = url
        new['pubDate'] = pubdate
        new['description'] = description

        return new

In [40]:
def test(df):
    for index in df.index:
        url = df['postUrl'][index]
        pubdate = df['pubDate'][index]
        description = df['description'][index]
        driver.get(url)
        time.sleep(1.5)
        try:
            load = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,".MGdpg > button:nth-child(1)")))

            print("Found {}".format(str(load)))
            i = 0
            while load.is_displayed() and i < 10000000: #int(sys.argv[2])
                load.click()
                time.sleep(1.5)
                load = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,".MGdpg > button:nth-child(1)")))
                #load = driver.find_element_by_css_selector('.MGdpg > button:nth-child(1)')

                #driver.execute_script("window.scrollBy(0,-1000);")
                print("Found {}".format(str(load)))
                i += 1
        except Exception as e:
            print(e)
            pass
        if index == 0:
            new = get_comments(url,pubdate,description)
        else:
            new = new.append(get_comments(url,pubdate,description))
        
    new.to_csv('leehsienloong_general_comments.csv')
    return new
        
        

In [41]:
test(df)

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="cc142301-6f6d-4f94-81f3-56990dbde39b")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="2e50b8ca-8cf1-48c9-964e-39d95398bc76")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="2b0516c8-bd17-471f-8f7d-92d371214e95")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="300da6d5-13df-43cd-b2d2-9051bcaee9c5")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="67d66b79-2dc2-4120-b313-e9288bafab69")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="50bdbbb2-da32-43de-9f94-7ca27cb7b3b0")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", elemen

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="7126f4b6-d167-4040-9b7a-f35387d489fd")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="be7024c8-6a7d-46fe-af50-25cb97910c8b")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="bb126404-851a-49e9-ac07-31b0b64a787c")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="2b799eb1-a90c-4660-a3e5-d0138d83e87f")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="cac91ea6-a388-4f92-90f4-ed43b97af721")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="1c82e230-054f-48a5-81dc-cf171f316d94")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", elemen

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="9da014c2-1a46-492f-95ca-61e30fcd73ad")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="595f9e29-b2e5-4061-a5ed-7bede494acc1")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="14b6642d-231c-4446-95f3-885432481826")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="eae5d270-5c8e-4419-88e5-eed73f710272")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="5a02c3d3-fc9f-4901-a843-12cafe53bda0")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="9f0577fe-58fa-440a-a256-ab8a65c5f842")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", elemen

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="53cbcd70-99f5-48ab-b497-44938a32e63d")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="cc4c03ef-5300-4f06-a41c-df9d956a6ebc")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="4fbf7907-cceb-44b3-bbe0-821d665e0398")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="bc1984de-9dc2-43a9-a732-30e4a7497b1a")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="365b51c0-3cf9-4ca7-b150-c83a993139c9")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="2523d94d-cb6b-4dd0-9ec6-8b1a379dbc68")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", elemen

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="7f587064-d79f-4ab1-b9f3-e936133e5050")>
Message: 

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="448b5601-2821-472e-a4d0-adbabc7d3a6d")>
Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="57ed1a11-6cbf-4081-96bf-f0899c9951d8")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d34

...

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="a3a22058-0281-4835-919f-7322226f23cb")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="3351de27-fe09-47fb-80d4-5f5b4f1b3659")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="08c46dc9-6a3d-4b20-b124-9474ea184520")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="239d7b56-41be-452e-b560-5d629e464366")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="22b4e898-268b-4582-9251-2c49b2b51590")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="dcd29317-1a37-4d32-8f9e-97bf06a18997")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", elemen

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="dbd6e2aa-d3af-4139-83d2-ebf01c65a566")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="85f6a4fc-d51f-4b6c-b6c8-097e117fa205")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="b33331aa-1ab4-4cf8-99ff-9cfe3741de27")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="466334eb-6804-49e5-a381-1d7c9759c3a3")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="2879775f-df1b-41b1-a39f-d3e076772822")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="cc5e082e-29ff-40c7-85ef-a6b453202ef3")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", elemen

Message: 

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="66b4e65f-7ea2-45d4-8003-9bf60ccb1918")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="972d407a-6f7b-48d7-bcc9-d944d3c0a21b")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="a8943ecb-5f9f-4ae0-a1a0-39194f6cac05")>
Message: 

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="953fc16d-dfcc-4457-8f6a-364eaf633207")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="e922f525-c977-46f8-9d4e-031962ac2d29")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="7203ec12-4ca7-470b-922c-f7e7cc515fde")>
Message: 

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="65758c49-d054-4520-b807-687614649872")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="dc91ba99-e4a1-4984-a83c-8f007a6be7fe")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="80dbb568-4ea9-44cd-a1fd-9a4547a2c6f0")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="1ce44fb9-a07f-49d7-9385-acb2ada18582")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="cfe9b50d-f94a-4744-9e08-90bef1f003a7")>
Message: 

Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae00", element="6b7961f7-5918-4f1e-85a6-ecda11bcf170")>
Found <selenium.webdriver.remote.webelement.WebElement (session="1d345369476c33a0e0c81cd8dce7ae

,username,comments,timestamp,postUrl,pubDate,description
0,pammyrose6,Happy Chap Goh Mei PM Lee❤️,2021-02-26T10:31:12.000Z,https://www.instagram.com/p/CLwH2XBrZef/,2021-02-26T10:22:56.000Z,Today is the fifteenth and final day of Chines...
1,303lsg,👯🍻🥇🏆🏂🧸🙏🦕👯🏄🎁🌈🎅👑🌴元宵節快樂！,2021-02-26T10:30:26.000Z,https://www.instagram.com/p/CLwH2XBrZef/,2021-02-26T10:22:56.000Z,Today is the fifteenth and final day of Chines...
2,kevinkong_canton,元宵节快乐,2021-02-26T10:47:38.000Z,https://www.instagram.com/p/CLwH2XBrZef/,2021-02-26T10:22:56.000Z,Today is the fifteenth and final day of Chines...
3,negritaojos_claros,❤️,2021-02-26T11:13:02.000Z,https://www.instagram.com/p/CLwH2XBrZef/,2021-02-26T10:22:56.000Z,Today is the fifteenth and final day of Chines...
4,plimx__520,元霄节快乐 Mr PM LEE!,2021-02-26T10:50:20.000Z,https://www.instagram.com/p/CLwH2XBrZef/,2021-02-26T10:22:56.000Z,Today is the fifteenth and final day of Chines...
...,...,...,...,...,...,...
14,maryadele8,"Good day, Mr PM, Lee Hsien Loong...🌻💪🌻",2020-01-20T04:52:25.000Z,https://www.instagram.com/p/B7c0svcHU8Q/,2020-01-18T06:07:44.000Z,The government’s task has become more complex ...
15,maryadele8,"Mr PM, please take care of your health, you a...",2020-01-20T04:53:39.000Z,https://www.instagram.com/p/B7c0svcHU8Q/,2020-01-18T06:07:44.000Z,The government’s task has become more complex ...
16,tongtuckkwong,🙌❤️,2020-01-20T07:33:36.000Z,https://www.instagram.com/p/B7c0svcHU8Q/,2020-01-18T06:07:44.000Z,The government’s task has become more complex ...
17,wyimlai_,You are such an elegant man.,2020-01-20T08:13:16.000Z,https://www.instagram.com/p/B7c0svcHU8Q/,2020-01-18T06:07:44.000Z,The government’s task has become more complex ...
